In [413]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline

In [414]:
auto=pd.read_csv('Auto.csv',engine='python',encoding='949')

In [415]:
auto[auto['horsepower']=='?']

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
32,25.0,4,98.0,?,2046,19.0,71,1,ford pinto
126,21.0,6,200.0,?,2875,17.0,74,1,ford maverick
330,40.9,4,85.0,?,1835,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,?,2905,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,?,2320,15.8,81,2,renault 18i


In [416]:
df=auto[auto['horsepower']!='?']

# The Validation Set Approach

In [433]:
df['horsepower'] = pd.to_numeric(df['horsepower'])

C:\Users\Hanyuri\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [434]:
df_train,df_test=train_test_split(df,test_size=0.5,random_state=1)
print('train data size : {}'.format(df_train.shape))
print('test data size : {}'.format(df_test.shape))

train data size : (196, 9)
test data size : (196, 9)


In [439]:
lm=smf.ols(formula='mpg ~ horsepower',data=df_train).fit()
y_pred=lm.predict(df_test)
print(mean_squared_error(df_test['mpg'],y_pred).round(3))

24.802


In [440]:
poly=pd.DataFrame()
poly['mpg']=df['mpg']
poly['horsepower']=df['horsepower']
poly['horsepower2']=df['horsepower']*df['horsepower']
poly['horsepower3']=df['horsepower']*df['horsepower']*df['horsepower']

In [441]:
poly_x=poly[['horsepower','horsepower2']]
poly_y=poly[['mpg']]
x2_train,x2_test,y2_train,y2_test=train_test_split(poly_x,poly_y,test_size=0.5,random_state=1)

In [444]:
lm2=LinearRegression()
lm2.fit(x2_train,y2_train)
y2_pred=lm2.predict(x2_test)
print(mean_squared_error(y2_test,y2_pred).round(3))

18.848


In [447]:
poly_x=poly[['horsepower','horsepower2','horsepower3']]
poly_y=poly[['mpg']]
x3_train,x3_test,y3_train,y3_test=train_test_split(poly_x,poly_y,test_size=0.5,random_state=1)

In [449]:
lm3=LinearRegression()
lm3.fit(x3_train,y3_train)
y3_pred=lm3.predict(x3_test)
print(mean_squared_error(y3_test,y3_pred).round(3))

18.805


# Leave-One-Out Cross-Validation

In [452]:
formula='mpg ~ horsepower'
glm = smf.glm(formula = formula, data=df).fit()
print(glm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    mpg   No. Observations:                  392
Model:                            GLM   Df Residuals:                      390
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                          24.066
Method:                          IRLS   Log-Likelihood:                -1178.7
Date:                Tue, 04 Aug 2020   Deviance:                       9385.9
Time:                        01:20:57   Pearson chi2:                 9.39e+03
No. Iterations:                     3   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.530      41.342
horsepower    -0.1578      0.006    -24.489      0.0

In [453]:
lm=smf.ols(formula='mpg ~ horsepower',data=df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Tue, 04 Aug 2020   Prob (F-statistic):           7.03e-81
Time:                        01:21:01   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.0

In [454]:
p_order = np.arange(1,6)
r_state = np.arange(0,10)

# LeaveOneOut CV
lm = LinearRegression()
loo = LeaveOneOut()
loo.get_n_splits(df)
scores = list()

for i in p_order:
    poly = PolynomialFeatures(i)
    x_poly = poly.fit_transform(df['horsepower'].values.reshape(-1,1))
    score = cross_val_score(lm, x_poly, df['mpg'], cv=loo, scoring='neg_mean_squared_error').mean()
    print(-score.round(2))

24.23
19.25
19.33
19.42
19.03


# k-fold

In [455]:
p_order = np.arange(1,11)

lm = LinearRegression()


for i in p_order:
    poly = PolynomialFeatures(i)
    x_poly = poly.fit_transform(df['horsepower'].values.reshape(-1,1))
    score = cross_val_score(lm, x_poly, df['mpg'], cv=10, scoring='neg_mean_squared_error').mean()
    print(-score.round(2))

27.44
21.24
21.34
21.35
20.91
20.8
20.95
21.08
21.04
20.98


# The Bootstrap

In [456]:
data=pd.DataFrame()
data['mpg']=df['mpg']
data['horsepower']=df['horsepower']

In [457]:
def boot(data,i):
    df=data.sample(n=i,replace=True,random_state=1)
    lm=smf.ols(formula='mpg ~ horsepower',data=df).fit()
    print(lm.params.round(3))
    return lm.summary()
    

In [458]:
boot(data,392)

Intercept     39.658
horsepower    -0.156
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     581.8
Date:                Tue, 04 Aug 2020   Prob (F-statistic):           2.52e-79
Time:                        01:22:07   Log-Likelihood:                -1174.0
No. Observations:                 392   AIC:                             2352.
Df Residuals:                     390   BIC:                             2360.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.6585      0.714     55.575      0.000      38.255      41.061
horsepower    -0.1559      0.006    -24.120      0.000      -0.169      -0.143
==============================================================================
Omnibus:                       17.579   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.965
Skew:                           0.450   Prob(JB):                     4.62e-05
Kurtosis:                       3.643   Cond. No.                         322.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [459]:
boot(data,1000)

Intercept     40.436
horsepower    -0.161
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     1520.
Date:                Tue, 04 Aug 2020   Prob (F-statistic):          8.59e-203
Time:                        01:22:17   Log-Likelihood:                -3022.3
No. Observations:                1000   AIC:                             6049.
Df Residuals:                     998   BIC:                             6058.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.4359      0.459     88.181      0.000      39.536      41.336
horsepower    -0.1608      0.004    -38.990      0.000      -0.169      -0.153
==============================================================================
Omnibus:                       29.569   Durbin-Watson:                   2.054
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.545
Skew:                           0.412   Prob(JB):                     1.41e-07
Kurtosis:                       3.277   Cond. No.                         324.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [468]:
def boot2(data,i):
    df=data.sample(n=i,replace=True,random_state=1)
    df['horsepower2']=df['horsepower']*df['horsepower']
    lm=smf.ols(formula='mpg ~ horsepower+horsepower2',data=df).fit()
    print(lm.params.round(3))
    return lm.summary()

In [469]:
boot2(data,1000)

Intercept      57.645
horsepower     -0.473
horsepower2     0.001
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     1045.
Date:                Tue, 04 Aug 2020   Prob (F-statistic):          2.02e-245
Time:                        01:29:33   Log-Likelihood:                -2919.9
No. Observations:                1000   AIC:                             5846.
Df Residuals:                     997   BIC:                             5861.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      57.6446      1.216     47.399      0.000      55.258      60.031
horsepower     -0.4731      0.021    -22.439      0.000      -0.515      -0.432
horsepower2     0.0012   8.28e-05     15.050      0.000       0.001       0.001
==============================================================================
Omnibus:                       48.946   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.907
Skew:                           0.296   Prob(JB):                     1.66e-23
Kurtosis:                       4.472   Cond. No.                     1.35e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.35e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""